# Part One
## Web Scraping and Data Cleaning

I known you can read the url directly from Pandas, but let's practice using Beautiful Soup. We'll use requests to get the html code from the url, then use Beatuiful Soup to find the table on the page. We'll then pass that table to pandas to make a dataframe

In [1]:
# import dependencies
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import pandas as pd
import numpy as np

In [2]:
# define function to get html info using requests
def get_html(url):
    '''url and returns a beautiful soup object for that webpage'''
    r = requests.get(url)
    doc = r.content
    soup = BeautifulSoup(doc, 'lxml')
    return soup

In [3]:
# call function to 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [4]:
# read html into bs object
soup = get_html(url)
# print(soup.prettify())

In [5]:
# get just the table on the wikipedia page
table = soup.find('table')
# print(table.prettify())

In [6]:
# cast the bs object for the table into a str so pandas can read it, then convert to df
df = pd.read_html(str(table), header=0)[0] # read_html returns a list of dfs. there should only be 1 table here, so index 0 should be the correct one
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data Cleaning
Criteria:
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

#### Data Exploration
Let's expore the data a little so we can see what we need to do. How many Boroughs are 'Not Assigned'? How many post codes have multiple neighborhoods?

In [7]:
# use .info() method to see how much data we have and if check for null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 3 columns):
Postcode         287 non-null object
Borough          287 non-null object
Neighbourhood    287 non-null object
dtypes: object(3)
memory usage: 6.9+ KB


In [8]:
# how many not assigned vlaues are there?
df['Borough'].value_counts()

Not assigned        77
Etobicoke           44
North York          38
Scarborough         37
Downtown Toronto    36
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         2
Mississauga          1
Name: Borough, dtype: int64

Let's drop all the rows that have a 'Not Assigned' Borrough

In [9]:
# filter out any rows where the Borough column is 'Not assigned'
df = df[df['Borough'] != 'Not assigned']
df['Borough'].value_counts() # verify value counts

Etobicoke           44
North York          38
Scarborough         37
Downtown Toronto    36
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Queen's Park         2
Mississauga          1
Name: Borough, dtype: int64

Next, let's find the rows where the neighbourhoods are 'Not assigned'. Once we've found those rows, we can set the 'Neighborhood' column equal to the 'Borough' column

In [10]:
# Find boroughs where the neighborhood is not assigned
no_neighborhood = df[df['Neighbourhood'] == 'Not assigned']
no_neighborhood

,Postcode,Borough,Neighbourhood
7,M7A,Queen's Park,Not assigned


Now that we know what rows to change, we can slice those rows and set the 'Neighbourhood' equal to the 'Borough'. There's technically only one row that we have to change for this data set, but this code would work in case there were multiple rows. It's good for practice.

In [11]:
# Find the indices of the desired rows
indices_to_change = no_neighborhood.index
# slice and change df
df.loc[indices_to_change, 'Neighbourhood'] = df.loc[indices_to_change, 'Borough']

# Verify Change
df[df['Postcode'] == "M7A"]

,Postcode,Borough,Neighbourhood
7,M7A,Queen's Park,Queen's Park


In [12]:
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Queen's Park,Queen's Park
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


The last step of the cleaning is to concactenate all neighbourhoods in each borough into one row. We can group by Postcode and then Borugh, then join each neighborhood in heaf group separated by ', '

In [13]:
# Group the df and get the series for 'Neighborhood'. Then join each str, separated vy ', '. Convert the new series to a df afterwards
clean_df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).to_frame()

# reset index
clean_df.reset_index(inplace=True)

clean_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [14]:
# display shape of df
clean_df.shape

(103, 3)

# Part Two
## Geocoding
We need to get coordinates (lattitude and longtitude) for each postal code. Geocoder wasn't working for me, and I tried other sources like geopy and mapbox with mixed results. Map box, was sucessful, but gave slightly different results from the provided csv. For grading simplicity and continuity, I'm going to use the csv.

After reding the data from the provided csv, Let's build a dataframe of coordinates for each unique postal code, then merge that df with the previously made df containing the borroughs and neighborhoods.

In [15]:
csv_path = 'https://cocl.us/Geospatial_data'
coordinate_df = pd.read_csv(csv_path)
coordinate_df.rename(columns={'Postal Code':'Postcode'}, inplace=True) # rename column so it matches previous df
coordinate_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
coordinate_df.shape

(103, 3)

In [17]:
# Merge dataframes
df = pd.merge(clean_df, coordinate_df, on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [18]:
df.shape

(103, 5)

# Part 3
## Foursquare API call and clustering

Let's filter for only the top 25% most populated postal codes in Toronto. To do this, let's scrape more data from the 2016 Canadian census. We can join this data on the 'Postcode' column to get the population for each postcode.

In [19]:
census_data = 'https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Table.cfm?Lang=Eng&T=1201&SR=1&S=22&O=A&RPP=9999&PR=0'

In [20]:
# read the census page into BS object from function in part 1
census_soup = get_html(census_data)

# print(census_soup.prettify())

In [21]:
# find the table in the page
census_table = census_soup.find('table')

# print(census_table.prettify())

In [22]:
# read table into df
census_df = pd.read_html(str(census_table), header=0, skiprows=[1])[0]
census_df.rename(columns={'Geographic name':'Postcode'}, inplace=True)
census_df.head()

,Postcode,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,CanadaFootnote 1,35151728,15412443,14072079
1,A0A,46587,26155,19426
2,A0B,19792,13658,8792
3,A0C,12587,8010,5606
4,A0E,22294,12293,9603


In [23]:
# Inner Join dfs on postcode = Geographic name. This will leave us with only the postcodes in Toronto
hood_pop_df = pd.merge(df, census_df, on='Postcode')
hood_pop_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,66108,20957,20230
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,35626,11588,11274
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,46943,17637,17161
3,M1G,Scarborough,Woburn,43.770992,-79.216917,29690,10116,9767
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,24383,9274,8985


In [24]:
# Calculate the top n % of neighborhoods
n = 0.25

# Get the index of the last row in the top n %
i = int(hood_pop_df.shape[0] * n)

# sort the hood_pop_df by population
hood_pop_df.sort_values(['Population, 2016'], ascending=False, inplace=True)

# slice the df at i for the top n percent
top_df = hood_pop_df.iloc[:i]
top_df.reset_index(inplace=True, drop=True)
top_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,M2N,North York,Willowdale South,43.770120,-79.408493,75897,36123,33765
1,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,66108,20957,20230
2,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,58293,22524,21876
3,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437,55959,17590,16808
4,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St...",43.815252,-79.284577,54680,16449,16092


In [25]:
top_df.shape

(25, 8)

In [26]:
# build query for foursquare api
# input api keys for now
client = input('Input Client key: ')
client_secret = input('Input Client Secret: ')

Input Client key: UZXIPYQ2CSFLPXBDCQTUKIETXYTK1EXJWSEMELCVDHSEZOOF
Input Client Secret: 1ERBZHULD03ZGTO3EKVYGIUVWV04Z2I1G2GQQTNOSBWWNPXB


In [27]:
top_df.dtypes

Postcode                                                object
Borough                                                 object
Neighbourhood                                           object
Latitude                                               float64
Longitude                                              float64
Population, 2016                                         int64
Total private dwellings, 2016                            int64
Private dwellings occupied by usual residents, 2016      int64
dtype: object

In [53]:
# Build Url
# version = 20180605
version = 20191114
radius = 400
limit = 100
base_uri = 'https://api.foursquare.com/v2/venues/explore'
postcode = top_df.loc[0,'Postcode']
lat = top_df.loc[0,'Latitude'].round(2)
long = top_df.loc[0,'Longitude'].round(2)
ll = f'{lat},{long}'

print(ll)

url = f'{base_uri}?client_id={client}&client_secret={client_secret}&v={version}&ll={ll}&radius={radius}&limit={limit}'
print(url)

43.77,-79.41
https://api.foursquare.com/v2/venues/explore?client_id=UZXIPYQ2CSFLPXBDCQTUKIETXYTK1EXJWSEMELCVDHSEZOOF&client_secret=1ERBZHULD03ZGTO3EKVYGIUVWV04Z2I1G2GQQTNOSBWWNPXB&v=20191114&ll=43.77,-79.41&radius=400&limit=100


In [54]:
# request data
response = requests.get(url)
r = response.json()

import json
print(json.dumps(r, indent=3, sort_keys=True))

{
   "meta": {
      "code": 200,
      "requestId": "5dcf2d096001fe001b06429e"
   },
   "response": {
      "groups": [
         {
            "items": [
               {
                  "reasons": {
                     "count": 0,
                     "items": [
                        {
                           "reasonName": "globalInteractionReason",
                           "summary": "This spot is popular",
                           "type": "general"
                        }
                     ]
                  },
                  "referralId": "e-0-4ae257cff964a520758d21e3-0",
                  "venue": {
                     "categories": [
                        {
                           "icon": {
                              "prefix": "https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_",
                              "suffix": ".png"
                           },
                           "id": "4bf58dd8d48988d118951735",
                          

In [55]:
# from the response, the data we want is in ['response']['groups'][0]['items']
venues = r['response']['groups'][0]['items'] # [0]['venue']
print(json.dumps(venues, indent=3, sort_keys=True))

[
   {
      "reasons": {
         "count": 0,
         "items": [
            {
               "reasonName": "globalInteractionReason",
               "summary": "This spot is popular",
               "type": "general"
            }
         ]
      },
      "referralId": "e-0-4ae257cff964a520758d21e3-0",
      "venue": {
         "categories": [
            {
               "icon": {
                  "prefix": "https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_",
                  "suffix": ".png"
               },
               "id": "4bf58dd8d48988d118951735",
               "name": "Grocery Store",
               "pluralName": "Grocery Stores",
               "primary": true,
               "shortName": "Grocery Store"
            }
         ],
         "id": "4ae257cff964a520758d21e3",
         "location": {
            "address": "5095 Yonge St",
            "cc": "CA",
            "city": "North York",
            "country": "Canada",
            "crossStreet": "in No

In [56]:
# print info about all the returned venues and what the type of venue
for venue in venues:
    v = venue['venue']
    
    # get the postal code from the returned data in case it differs from teh search criteria. We don't want one postal code getting credit for a different one.
    try: 
        full_pCode = v['location']['postalCode']
        # we only care about the first half of the postal code
        pCode = full_pCode[0:3]
    # If the postcode field isn't populated, fill with something else
    except: pCode = 'NA_'
    
    print(pCode + ' ' + v['name'] + ': ' + v['categories'][0]['name'])

M2N Loblaws: Grocery Store
M2N Cineplex Cinemas Empress Walk: Movie Theater
M2N Konjiki Ramen: Ramen Restaurant
NA_ Aroma Espresso Bar: Café
M2N Starbucks: Coffee Shop
NA_ Wako Sushi + Bar: Japanese Restaurant
M2N PetSmart: Pet Store
M2N Saryo: Café
M2N Michaels: Arts & Crafts Store
M2N Empress Walk: Shopping Mall
M2N KINTON RAMEN: Ramen Restaurant
M2N Booster Juice: Juice Bar
M2N Wendy's: Fast Food Restaurant
M2N Starbucks: Coffee Shop
M2N Mel Lastman Square: Plaza
M2N Symposium Cafe Restaurant & Lounge: Restaurant
NA_ Pho 88 Vietnamese Cuisine: Vietnamese Restaurant
M2N Ajisen Ramen 味千ラーメン: Ramen Restaurant
M2N Dollarama: Discount Store
M2N Aburi Room: Sushi Restaurant
M2N Dairy Queen (Treat): Ice Cream Shop
M2N Subway: Sandwich Place
NA_ Best Buy: Electronics Store
NA_ Ten Ren's Tea Time 喫茶新饌: Bubble Tea Shop
M2N Boston Pizza: Pizza Place
M2N Subway: Sandwich Place
M2N Freshslice Pizza: Pizza Place
M2N Second Cup: Coffee Shop
M2N Ichiban Sushi House: Sushi Restaurant
M2N Novotel Tor

In [44]:
venues[0]['venue']['location']['postalCode']

'M2N 6Z4'

After some preliminary investigation, I've found it's possible for venues in nearby postcodes to appear show up in the reccommended venues. This could also mean that a single venue will show up for multiple postcodes. 

I think this is OK. Even if a venue doesn't technically belong to a given postcode, it should count for that postcode's profile as long as the venue is within a specified radius of the postcode coordinates.

In [73]:
# define function to get the vendor info and process the data
def get_vendor_info(latitude, longitude, postcode, client_id, client_secret,  venue_list, radius=500, limit=100, version='20191115'):
    '''
    Takes the coordinates of a postcode and gets foursquare reccomendations for those coordinates. 
    
    Returns a list of dictionairies containing the name, category, and postcode for each returned venue
    '''
    # build url
    base_uri = 'https://api.foursquare.com/v2/venues/explore'
    ll = f'{latitude},{longitude}'
    url = f'{base_uri}?client_id={client_id}&client_secret={client_secret}&v={version}&ll={ll}&radius={radius}&limit={limit}'
    
    # make api request
    response = requests.get(url)
    r = response.json() # convert response to json
    
    # get the list of venues under the 'items' key
    venues = r['response']['groups'][0]['items']
    
    # loop through the returned venues, create a dictioary containing the poscode, venue name, and category of the venue, then append thsi dictionary to the specified list
    for venue in venues:
        venue_dict = {'Postcode':postcode,
                      'Venue name':venue['venue']['name'],
                      'Venue category':venue['venue']['categories'][0]['name']}
        
        # append the dictionary to the provided list
        venue_list.append(venue_dict)

In [74]:
top_df.columns

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude',
       'Population, 2016', 'Total private dwellings, 2016',
       'Private dwellings occupied by usual residents, 2016'],
      dtype='object')

In [75]:
# Call the get_vendor_info function for each postcode in the top_df
venues_list = [] # empty list to hold the returned data

# use iterate through the top df and get the data
for i in top_df.index:
    # get the required data from the df
    lat = top_df.loc[i, 'Latitude']
    long = top_df.loc[i, 'Longitude']
    pCode = top_df.loc[i, 'Postcode']
    
    # call out definied function
    print(f'Getting data for {pCode}...')
    get_vendor_info(lat, long, pCode, client, client_secret, venues_list)
    print('Done!')

Getting data for M2N...
Done!
Getting data for M1B...
Done!
Getting data for M2J...
Done!
Getting data for M9V...
Done!
Getting data for M1V...
Done!
Getting data for M5V...
Done!
Getting data for M1W...
Done!
Getting data for M1K...
Done!
Getting data for M1E...
Done!
Getting data for M4C...
Done!
Getting data for M1P...
Done!
Getting data for M6H...
Done!
Getting data for M6M...
Done!
Getting data for M3N...
Done!
Getting data for M6N...
Done!
Getting data for M5A...
Done!
Getting data for M6K...
Done!
Getting data for M2R...
Done!
Getting data for M9W...
Done!
Getting data for M6P...
Done!
Getting data for M3C...
Done!
Getting data for M9C...
Done!
Getting data for M6E...
Done!
Getting data for M8V...
Done!
Getting data for M1S...
Done!


In [76]:
df = pd.DataFrame(venues_list)
df.head()

,Postcode,Venue name,Venue category
0,M2N,Loblaws,Grocery Store
1,M2N,Konjiki Ramen,Ramen Restaurant
2,M2N,The Keg,Steakhouse
3,M2N,Cineplex Cinemas Empress Walk,Movie Theater
4,M2N,Aroma Espresso Bar,Café


In [78]:
# See how many venues were returned for each post code
df.groupby(['Postcode']).count()

,Venue name,Venue category
Postcode,,
M1B,1,1
M1E,8,8
M1K,4,4
M1P,6,6
M1S,5,5
M1V,2,2
M1W,14,14
M2J,64,64
M2N,34,34


In [81]:
# One hot encode the venue categories
df.set_index(['Postcode'], inplace=True)
one_hot_df = pd.get_dummies(df['Venue category'])

In [82]:
one_hot_df.head()

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
Postcode,,,,,,,,,,,,,,,,,,,,,
M2N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M2N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M2N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M2N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
M2N,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# calculate the frequency for each category
freq_df = one_hot_df.groupby(['Postcode']).mean()
freq_df

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
Postcode,,,,,,,,,,,,,,,,,,,,,
M1B,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000
M1E,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000
M1K,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000
M1P,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.166667,0.000000,0.0000
M1S,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000
M1V,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000
M1W,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.0000
M2J,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,...,0.000000,0.03125,0.000000,0.015625,0.015625,0.015625,0.000,0.000000,0.015625,0.0625
M2N,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,...,0.058824,0.00000,0.000000,0.000000,0.000000,0.000000,0.000,0.029412,0.000000,0.0000
